<a href="https://colab.research.google.com/github/suhbaca/AI4GOOD_TeamProject/blob/sasha/LLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#access huggingface
!huggingface-cli login

In [ ]:
# Load model directly

import transformers torch accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

In [ ]:
#prompt engineering

instruction= 'The user is trying to find a word that matches the description in the prompt. The answer is 5 possible words that the prompt is trying to describe. The following are examples of Prompt-Answer format.\n'  # eg Given the context provided, answer the question

few_shot = "Prompt: it’s a type of bird, kind of rare. They’re very colorful, with long tails, and they can mimic sounds. They live in tropical areas\n Answer:  Hummingbird, Parrot, Finch, Toucan, Macaw\n Prompt: It’s a type of bird, very colorful, and it can mimic human speech. People sometimes keep them as pets. They live in tropical areas.\nAnswer: Parrot, Macaw, Cockatoo, Canary, Finch\nPrompt: I need the...you know...the thing for writing\nAnswer: Pen, Pencil, Quill, Fountain Pen, Crayon\nPrompt : I want to go to the...um...place where we get groceries\n  Answer: Store, Supermarket, Market, Grocery, Shop\nPrompt : "

system_prompt = 'Now it is your turn to give the answer to the last prompt. Give 30 possible words being described instead of just 5 like in the examples. It is important that you give 30 words. It is very important that you only provide the final output without any additional comments or remarks or notes.'
  # You will be given the description of a word .....

In [ ]:
#if the user is submitting a prompt for the first time, there is no extra extra added prompt
#if the user reqyests for the words to be reloaded, in the back end the prompt is asked again and an extra prompt saying to omit the already generated words is used

In [ ]:
def get_prompt():
  #get user input from voice recog system

  prompt = ""
  return prompt

In [ ]:
def add_template(prompt):


  instruction= 'The user is trying to find a word that matches the description in the prompt. The answer is 5 possible words that the prompt is trying to describe. The following are examples of Prompt-Answer format.\n'  # eg Given the context provided, answer the question

  few_shot = "Prompt: it’s a type of bird, kind of rare. They’re very colorful, with long tails, and they can mimic sounds. They live in tropical areas\n Answer:  Hummingbird, Parrot, Finch, Toucan, Macaw\n Prompt: It’s a type of bird, very colorful, and it can mimic human speech. People sometimes keep them as pets. They live in tropical areas.\nAnswer: Parrot, Macaw, Cockatoo, Canary, Finch\nPrompt: I need the...you know...the thing for writing\nAnswer: Pen, Pencil, Quill, Fountain Pen, Crayon\nPrompt : I want to go to the...um...place where we get groceries\n  Answer: Store, Supermarket, Market, Grocery, Shop\nPrompt : "

  system_prompt = 'Now it is your turn to give the answer to the last prompt. Give 30 possible words being described instead of just 5 like in the examples. It is important that you give 30 words. It is very important that you only provide the final output without any additional comments or remarks or notes.'
  # You will be given the description of a word .....
  chat_prompt= instruction + few_shot + prompt + system_prompt
  return chat_prompt

In [ ]:
def create_pipeline():
  pipeline = transformers.pipeline(
  "text-generation",

  model=model,

  tokenizer=tokenizer,

  torch_dtype=torch.float16,

  device_map="auto",

  )
  return pipeline

SyntaxError: incomplete input (<ipython-input-3-53a2ac4d3340>, line 1)

In [ ]:
def get_suggestions(chat_prompt):
  sequences = pipeline(
  chat_prompt,

  do_sample=True,

  top_k=20,

  num_return_sequences=1,

  eos_token_id=tokenizer.eos_token_id,

  max_length=800,

  temperature= 0.3,

  return_full_text= False

  )



In [ ]:
class Chat:

  def __init__(self) :
    words_generated={}


  def create_chat(self):

